In [ ]:
%pip install -q google-colab-selenium
%pip install -q selenium

Code to scrape from apprentice job data and insert one row at a time into bigquery and be able to view in google sheets (Apprenticeship Data).

In [ ]:
from google.cloud import bigquery
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from datetime import datetime
import google_colab_selenium as gs

def scrape_jobs(occupation, project_id, dataset_id, table_id):
    # Initialize WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument('--disable-blink-features=AutomationControlled')  # Make Selenium less detectable
    driver = gs.Chrome(options=options)

    url = f"https://www.apprenticeship.gov/finder/listings?occupation={occupation.replace(' ', '%20')}&location="
    driver.get(url)

    try:
        # Load More button logic
        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom of the page
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".listings-cards-load-more-btn-container .btn.btn--secondary.load-more"))
                )
                load_more_button.click()
                time.sleep(5)  # Wait for new jobs to load
            except Exception as e:
                print(f"Load more button not found or not clickable: {e}")
                break

        # Wait for job listings to load
        jobs = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-test="component-listing-card"]'))
        )

        # Iterate over each job listing
        for i in range(len(jobs)):
            try:
                # Re-find job elements to avoid stale element reference
                jobs = driver.find_elements(By.CSS_SELECTOR, '[data-test="component-listing-card"]')
                job = jobs[i]
                time.sleep(5)

                # Extract title, program, and location
                title = job.find_element(By.CLASS_NAME, 'left-section_main-title').text
                program = job.find_element(By.CLASS_NAME, 'bottom-content_p-title').text
                location = job.find_element(By.CLASS_NAME, 'bottom-content_p-content').text

                # Click the job to open the details view
                job.click()

                # Wait for the job summary to load
                summary_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "description-section"))
                )
                summary = summary_element.text

                # Create a job dictionary
                job_listing = {
                    "job_title": title,
                    "company": program,
                    "location": location,
                    "summary": summary,
                    "scraped_occupation_title": occupation
                }

                # Immediately upload the job to BigQuery
                upload_to_bigquery(table_ref, job_listing)
                time.sleep(4)

            except Exception as e:
                print(f"Error extracting job details: {e}")
                continue
    finally:
        driver.quit()

def upload_to_bigquery(table_ref, job_listing):
    """Uploads a single job listing to a BigQuery table."""
    client = bigquery.Client()

    # Add the current timestamp to the job listing
    job_listing["inserted_at"] = datetime.utcnow().isoformat()  # Add UTC timestamp

    # Insert single job listing into the table
    errors = client.insert_rows_json(table_ref, [job_listing])  # Insert a single job listing
    if errors == []:
        print(f"Job successfully inserted into {table_ref.table_id}.")
    else:
        print(f"Errors occurred while inserting job: {errors}")

def ensure_table_exists(client, project_id, dataset_id, table_id):
    """Ensures the BigQuery table exists. Creates it if it doesn't."""
    table_ref = client.dataset(dataset_id).table(table_id)

    schema = [
        bigquery.SchemaField("job_title", "STRING"),
        bigquery.SchemaField("company", "STRING"),
        bigquery.SchemaField("location", "STRING"),
        bigquery.SchemaField("summary", "STRING"),
        bigquery.SchemaField("scraped_occupation_title", "STRING"),
        bigquery.SchemaField("inserted_at", "TIMESTAMP")
    ]

    try:
        table = client.get_table(table_ref)
        print(f"Table {table_id} already exists.")
    except Exception as e:
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)
        print(f"Created table {table_id}.")

    return table_ref

# List of occupations to scrape
occupations = ["Warehouse Associate"]  # done with energy analyst, product manager, HVAC Engineer, Nurse, and welder

# BigQuery details
project_id = "intern-sandbox-427618"
dataset_id = "apprenticeship_data"
table_id = "apprenticeship_jobs_data"

# Initialize BigQuery client and ensure table exists once
client = bigquery.Client(project=project_id)
table_ref = ensure_table_exists(client, project_id, dataset_id, table_id)

# Iterate over each occupation and scrape job listings
for occupation in occupations:
    scrape_jobs(occupation, project_id, dataset_id, table_id)


In [ ]:
#this works with bigquery, the data is scraped data first and then put into bigquery all together
from google.cloud import bigquery
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
import google_colab_selenium as gs

def scrape_jobs(occupation):
    # Initialize WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument('--disable-blink-features=AutomationControlled')  # Make Selenium less detectable
    driver = gs.Chrome(options=options)

    url = f"https://www.apprenticeship.gov/finder/listings?occupation={occupation.replace(' ', '%20')}&location="
    driver.get(url)

    # List to store the scraped data
    job_listings = []

    try:
        # Load More button logic
        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom of the page
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".listings-cards-load-more-btn-container .btn.btn--secondary.load-more"))
                )
                load_more_button.click()
                time.sleep(2)  # Wait for new jobs to load
            except Exception as e:
                print(f"Load more button not found or not clickable: {e}")
                break

        # Wait for job listings to load
        jobs = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-test="component-listing-card"]'))
        )

        # Iterate over each job listing
        for i in range(len(jobs)):
            try:
                # Re-find job elements to avoid stale element reference
                jobs = driver.find_elements(By.CSS_SELECTOR, '[data-test="component-listing-card"]')
                job = jobs[i]
                time.sleep(2)

                # Extract title, program, and location
                title = job.find_element(By.CLASS_NAME, 'left-section_main-title').text
                program = job.find_element(By.CLASS_NAME, 'bottom-content_p-title').text
                location = job.find_element(By.CLASS_NAME, 'bottom-content_p-content').text

                # Click the job to open the details view
                job.click()

                # Wait for the job summary to load
                summary_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "description-section"))
                )
                summary = summary_element.text

                # Append the job details to the list
                job_listings.append({
                    "job_title": title,
                    "company": program,
                    "location": location,
                    "summary": summary,
                    "scraped_occupation_title": occupation
                })
                time.sleep(1)

            except Exception as e:
                print(f"Error extracting job details: {e}")
                continue
    finally:
        driver.quit()

    return job_listings

def upload_to_bigquery(project_id, dataset_id, table_id, job_listings):
    """Uploads data to a BigQuery table."""
    client = bigquery.Client(project=project_id)

    # Define table schema
    schema = [
        bigquery.SchemaField("job_title", "STRING"),
        bigquery.SchemaField("company", "STRING"),
        bigquery.SchemaField("location", "STRING"),
        bigquery.SchemaField("summary", "STRING"),
        bigquery.SchemaField("scraped_occupation_title", "STRING")
    ]

    # Create the table if it doesn't exist
    table_ref = client.dataset(dataset_id).table(table_id)
    try:
        table = client.get_table(table_ref)
        print(f"Table {table_id} already exists.")
    except Exception as e:
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Created table {table_id}.")

    # Insert data into the table
    errors = client.insert_rows_json(table_ref, job_listings)
    if errors == []:
        print(f"Data successfully inserted into {table_id}.")
    else:
        print(f"Errors occurred while inserting data: {errors}")

# List of occupations to scrape
occupations = ["Product Manager"]

# BigQuery details
project_id = "intern-sandbox-427618"
dataset_id = "apprenticeship_data"
table_id = "apprenticeship_jobs_data"

# Iterate over each occupation and scrape job listings
for occupation in occupations:
    job_listings = scrape_jobs(occupation)
    upload_to_bigquery(project_id, dataset_id, table_id, job_listings)


In [ ]:
from google.cloud import bigquery

# Replace with your Google Cloud project ID and dataset ID
project_id = "intern-sandbox-427618"
dataset_id = "apprenticeship_data"

# Initialize a BigQuery client
client = bigquery.Client(project=project_id)

# Define the table ID
table_id = f"{project_id}.{dataset_id}.apprenticeship_jobs_data"

# Define the schema for the table
schema = [
    bigquery.SchemaField("job_title", "STRING"),
    bigquery.SchemaField("company", "STRING"),
    bigquery.SchemaField("location", "STRING"),
    bigquery.SchemaField("summary", "STRING"),
    bigquery.SchemaField("scraped_occupation_title", "STRING")
]


# Create the table object
table = bigquery.Table(table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  # API request
print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")


In [ ]:
from google.cloud import bigquery

# Replace with your Google Cloud project ID and dataset ID
project_id = "intern-sandbox-427618"
dataset_id = "apprenticeship_data"
table_id = f"{project_id}.{dataset_id}.apprenticeship_jobs_data"

# Initialize a BigQuery client
client = bigquery.Client(project=project_id)

# Get the current table object
table = client.get_table(table_id)

# Define the new schema field
new_schema_field = bigquery.SchemaField("inserted_at", "TIMESTAMP")

# Add the new field to the existing schema
new_schema = list(table.schema)  # Copy the existing schema
new_schema.append(new_schema_field)  # Append the new column

# Update the table object with the new schema
table.schema = new_schema

# Update the table in BigQuery
table = client.update_table(table, ["schema"])  # API request

# Print confirmation
print(f"Table {table.project}.{table.dataset_id}.{table.table_id} schema updated successfully.")


In [ ]:
#Without bigquery and with bucket storage

from google.cloud import storage
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import google_colab_selenium as gs


def scrape_jobs(occupation):
    # Initialize WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    # driver = webdriver.Chrome(options=options)
    options.add_argument('--disable-blink-features=AutomationControlled')  # Make Selenium less detectable
    driver = gs.Chrome(options=options)

    url = f"https://www.apprenticeship.gov/finder/listings?occupation={occupation.replace(' ', '%20')}&location="
    driver.get(url)

    # CSV file setup
    source_file_name = f'job_listings_{occupation.replace(" ", "_")}.csv'
    with open(source_file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(["Title", "Job", "Location", "Summary", "scraped_occupation_title"])

        try:
            # Load More button logic
            while True:
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom of the page
                    load_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, ".listings-cards-load-more-btn-container .btn.btn--secondary.load-more"))
                    )
                    load_more_button.click()
                    time.sleep(2)  # Wait for new jobs to load
                except Exception as e:
                    print(f"Load more button not found or not clickable: {e}")
                    break

            # Wait for job listings to load
            jobs = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-test="component-listing-card"]'))
            )

            # Iterate over each job listing
            for i in range(len(jobs)):
                try:
                    # Re-find job elements to avoid stale element reference
                    jobs = driver.find_elements(By.CSS_SELECTOR, '[data-test="component-listing-card"]')
                    job = jobs[i]
                    time.sleep(2)

                    # Extract title, program, and location
                    title = job.find_element(By.CLASS_NAME, 'left-section_main-title').text
                    program = job.find_element(By.CLASS_NAME, 'bottom-content_p-title').text
                    location = job.find_element(By.CLASS_NAME, 'bottom-content_p-content').text

                    # Click the job to open the details view
                    job.click()

                    # Wait for the job summary to load
                    summary_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "description-section"))
                    )
                    summary = summary_element.text

                    # Write to the CSV file
                    writer.writerow([title, program, location, summary, occupation])
                    time.sleep(1)

                except Exception as e:
                    print(f"Error extracting job details: {e}")
                    continue
        finally:
            driver.quit()

    return source_file_name

def upload_to_gcp_bucket(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the GCP bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# List of occupations to scrape
occupations = ["Energy Analyst"]

# GCP bucket name
bucket_name = "scrapingdata-harshit"

# Iterate over each occupation and scrape job listings
for occupation in occupations:
    source_file_name = scrape_jobs(occupation)
    destination_blob_name = f"RAPIDS-Jobs-Data/job_listings_{occupation.replace(' ', '_')}.csv"
    upload_to_gcp_bucket(bucket_name, source_file_name, destination_blob_name)
